In [1]:
import glob
import numpy as np
import pre
import matplotlib.pyplot as plt
import numpy as np
import loc

def s(im):
    plt.imshow(im,cmap="gray")
    plt.show()

In [2]:
folders_label =[("./answer_images/0/*",[1,0,0,0,0,0]),
                ("./answer_images/1/*",[0,1,0,0,0,0]),
                ("./answer_images/2/*",[0,0,1,0,0,0]),
                ("./answer_images/3/*",[0,0,0,1,0,0]),
                ("./answer_images/4/*",[0,0,0,0,1,0]),
                ("./answer_images/5/*",[0,0,0,0,0,1])]


In [3]:
paths = []
labels = []
for path,label in folders_label:
    new_paths = glob.glob(path)
    labels = labels + ([label] * len(new_paths))
    paths = paths + new_paths

In [4]:
x = []
for path in paths:
    x.append([pre.imread(path=path) / 255])
    
x = np.array(x, dtype = np.float64 )
y = np.array(labels, dtype = np.float64)

In [5]:
x.shape

(360, 1, 33, 174)

In [6]:
import nets
net = nets.answer_model()
net.load_weights("./answer_weights.pickle")

In [8]:
net.numpy_train(x,y, epochs=100)

0
Variable containing:
 0.1155
[torch.FloatTensor of size 1]

1
Variable containing:
1.00000e-02 *
  9.6354
[torch.FloatTensor of size 1]

2
Variable containing:
 0.1077
[torch.FloatTensor of size 1]

3
Variable containing:
1.00000e-02 *
  8.1261
[torch.FloatTensor of size 1]

4
Variable containing:
1.00000e-02 *
  9.5294
[torch.FloatTensor of size 1]

5
Variable containing:
1.00000e-02 *
  7.4736
[torch.FloatTensor of size 1]

6
Variable containing:
1.00000e-02 *
  8.9694
[torch.FloatTensor of size 1]

7
Variable containing:
1.00000e-02 *
  7.4544
[torch.FloatTensor of size 1]

8
Variable containing:
1.00000e-02 *
  6.8768
[torch.FloatTensor of size 1]

9
Variable containing:
1.00000e-02 *
  5.4439
[torch.FloatTensor of size 1]

10
Variable containing:
1.00000e-02 *
  5.4589
[torch.FloatTensor of size 1]

11
Variable containing:
1.00000e-02 *
  5.6984
[torch.FloatTensor of size 1]

12
Variable containing:
1.00000e-02 *
  5.1885
[torch.FloatTensor of size 1]

13
Variable containing:
1.

In [9]:
net.save_weights("answer_weights.pickle")